# Step 1: Install Dependencies

In [1]:
!pip install python-dotenv neo4j openai tiktoken langchain langchain-openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59


# Step 2: Load Environment Variables

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from dotenv import load_dotenv
import os

env_path = '/content/drive/MyDrive/rag2.env'
load_dotenv(env_path)

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE") or "neo4j"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ENDPOINT = os.getenv("OPENAI_BASE_URL") + "/embeddings"

# Constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'TextChunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

Mounted at /content/drive


# Step 3: Connect to Neo4j

In [3]:
from langchain_community.graphs import Neo4jGraph

kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

# Check how many chunks exist
kg.query("MATCH (c:TextChunk) RETURN count(c) AS count")

<ipython-input-3-c4e08c32f350>:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  kg = Neo4jGraph(


[{'count': 1346}]

# Step 4: Compute Embeddings

In [4]:
kg.query("""
MATCH (chunk:TextChunk) WHERE chunk.textEmbedding IS NULL
WITH chunk, genai.vector.encode(
  chunk.text,
  "OpenAI",
  {
    token: $openAiApiKey,
    endpoint: $openAiEndpoint
  }) AS vector
CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", vector)
""", params={
    "openAiApiKey": OPENAI_API_KEY,
    "openAiEndpoint": OPENAI_ENDPOINT
})

[]

# Step 5: Search Function to Preview Similar Chunks

In [11]:
def neo4j_vector_search(question, top_k=5):
    query = """
    WITH genai.vector.encode(
      $question,
      "OpenAI",
      {
        token: $openAiApiKey,
        endpoint: $openAiEndpoint
      }) AS question_embedding
    CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) YIELD node, score
    RETURN score, node.text AS text, node.company AS company, node.year AS year
    ORDER BY score DESC
    """
    return kg.query(query, params={
        'question': question,
        'openAiApiKey': OPENAI_API_KEY,
        'openAiEndpoint': OPENAI_ENDPOINT,
        'index_name': VECTOR_INDEX_NAME,
        'top_k': top_k
    })

# Example usage
results = neo4j_vector_search("What is Walmart's business model?", top_k=10)
for r in results:
    print(f"[{r['company']} {r['year']}] ({r['score']:.3f}) → {r['text'][:150]}...")

[WMT 2025] (0.928) → ITEM 1. BUSINESS General Walmart Inc. ("Walmart," the "Company" or "we") is a people-led, technology-powered omni-channel retailer dedicated to helpin...
[WMT 2025] (0.922) → and eCommerce innovation, which enables us to leverage technology, talent and expertise, and expand our assortment and service offerings. We are enhan...
[WMT 2025] (0.922) → two major categories: retail and wholesale. These categories consist of many formats, including: supercenters, supermarkets, warehouse clubs (includin...
[WMT 2025] (0.921) → stores and eCommerce, such as through our pickup and delivery services in most of our markets, including same-day delivery. We continue to expand our ...
[WMT 2025] (0.919) → for brands and online marketplace sellers, supply chain and fulfillment capabilities to online marketplace sellers, and data analytics and insights fo...
[WMT 2025] (0.919) → national and global suppliers the opportunity to grow with us. • Sustainability . Walmart's sustainabili

In [12]:
def neo4j_vector_search_batch(questions, top_k=5):
    all_results = {}
    for question in questions:
        query = """
        WITH genai.vector.encode(
          $question,
          "OpenAI",
          {
            token: $openAiApiKey,
            endpoint: $openAiEndpoint
          }) AS question_embedding
        CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) YIELD node, score
        RETURN score, node.text AS text, node.company AS company, node.year AS year
        """
        results = kg.query(query, params={
            'question': question,
            'openAiApiKey': OPENAI_API_KEY,
            'openAiEndpoint': OPENAI_ENDPOINT,
            'index_name': VECTOR_INDEX_NAME,
            'top_k': top_k
        })
        all_results[question] = results
    return all_results

In [15]:
questions = [
    "What is Apple's business model?",
    "What risks are mentioned by Tesla?",
    "How does Microsoft handle competition?"
]

results = neo4j_vector_search_batch(questions, top_k=5)

for question, answers in results.items():
    print(f"\n--- Question: {question} ---")
    for r in answers:
        print(f"[{r.get('company')} {r.get('year')}] ({r['score']:.3f}) → {r['text']}")


--- Question: What is Apple's business model? ---
[AAPL 2024] (0.917) → Item 1. Business Company Background The Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September. Products iPhone iPhone ® is the Company’s line of smartphones based on its iOS operating system. The iPhone line includes iPhone 16 Pro, iPhone 16, iPhone 15, iPhone 14 and iPhone SE ® . Mac Mac ® is the Company’s line of personal computers based on its macOS ® operating system. The Mac line includes laptops MacBook Air ® and MacBook Pro ® , as well as desktops iMac ® , Mac mini ® , Mac Studio ® and Mac Pro ® . iPad iPad ® is the Company’s line of multipurpose tablets based on its iPadOS ® operating system. The iPad line includes iPad Pro ® , iPad Air ® , iPad and iPad mini ® . Wearables, Home and Accessories Wearables includes

# Step 6: Setup LangChain GraphRAG

In [16]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Neo4jVector
from langchain.chains import RetrievalQAWithSourcesChain
import textwrap

# Create a vector store from the graph
vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

retriever = vector_store.as_retriever()

# Setup RAG chain
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY),
    chain_type="stuff",
    retriever=retriever
)

# Step 7: Ask Questions with LLM

In [17]:
def prettychain(question: str):
    response = qa_chain({"question": question}, return_only_outputs=True)
    print("\nAnswer:\n" + textwrap.fill(response["answer"], width=80))

In [19]:
# Ask real questions
prettychain("What is Netflix's primary business?")


Answer:
Netflix's primary business is providing entertainment services through streaming
content to its members.


In [20]:
prettychain("Where is Apple headquartered?")


Answer:
Apple is headquartered in Cupertino, California, United States.


In [21]:
prettychain("What are the top risks mentioned in Johnson & Johnson's 10-K?")


Answer:
The top risks mentioned in Johnson & Johnson's 10-K include legal proceedings,
intellectual property rights challenges, and product reliability concerns.


In [24]:
prettychain("Where are the primary suppliers for Tesla?")


Answer:
Tesla's primary suppliers for battery cells are Panasonic and Contemporary
Amperex Technology Co. Limited (CATL).


In [27]:
prettychain("How is ExxonMobil addressing climate change and the energy transition?")


Answer:
ExxonMobil is addressing climate change and the energy transition by
establishing a Low Carbon Solutions (LCS) business unit and continuing efforts
in existing businesses to advance the development and deployment of technologies
and projects such as CCS, hydrogen, lower-emission fuels, and lithium. They are
also collaborating with universities and commercial partners to drive
breakthroughs in cancer, Alzheimer’s, diabetes, and more. ExxonMobil aims to
play a leading role in the energy transition and achieve net zero emissions with
the support of technology advancements and government policies.


## 📊 Stage 4(2):  Batch GraphRAG System Implementation Summary

### 🎯 Objective
To extend the GraphRAG system into a **scalable, batch-capable prototype**, allowing multi-company 10-K filings to be ingested, structured as a Neo4j knowledge graph, embedded with vector search capabilities, and queried via an LLM through a Retrieval-Augmented Generation (RAG) pipeline.

---

### ✅ Key Components and What Was Implemented

1. **🔌 Neo4j Connection**
   - Connected to a Neo4j AuraDB instance with :TextChunk nodes created from **multiple companies’** latest 10-K filings (e.g., AAPL, JPM, XOM).
   -	Each node contains structured metadata: company, year, section, seq, source, cik, and cusip6.

2. **🧱 Node & Relationship Construction (Batch Mode)**
   - Extracted and chunked 10-K sections (item1, item1a, item7, item7a) using RecursiveCharacterTextSplitter.
   - Created :TextChunk nodes and :Entity nodes (e.g., organizations, products, risks).
   - Established relationships:
     - [:MENTIONS] between chunks and entities (from NER or Phrase Matching)
     - [:CO_OCCURS_WITH] between co-mentioned entities (limited to N ≤ 5 per chunk to control explosion)
   - Added company metadata to entity-level relationships for filtered subgraph analysis.

3. **📐 Embedding & Vector Index**
   - Used OpenAI embeddings to compute semantic vectors for all TextChunk.text.
   - Stored vectors as textEmbedding property and created a Neo4j vector index: form_10k_chunks.

4. **🔍 Semantic Retrieval**
   - Implemented neo4j_vector_search() to:
    - Embed a user query
    - Search top-k nearest chunks from the vector index
    - Return relevant paragraph texts for grounding answers.

5. **🤖 RAG Integration (LangChain)**
   -	Created Neo4jVector retriever with TextChunk nodes.
   - Connected OpenAI’s ChatGPT model using LangChain’s RetrievalQAWithSourcesChain.
   - Implemented prettychain() to ask complex business questions grounded in retrieved SEC filing text.

---

### 💬 Example Multi-Company Questions Supported

- "What is Netflix's primary business?"
- "Where is Apple headquartered?"
- "What are the top risks mentioned in Johnson & Johnson's 10-K?"
- "Where are the primary suppliers for Tesla?"
- "How is ExxonMobil addressing climate change and the energy transition?"

---

### ✅ Outcome
This stage delivered a **multi-company GraphRAG pipeline**, which:
- Ingests multiple firms' 10-K filings  
- Extracts paragraph- and entity-level structure  
- Computes and stores semantic embeddings  
- Enables contextual retrieval and LLM-based question answering

> It lays the groundwork for scalable, explainable financial document QA systems.
